In [1]:
# !pip install -i https://test.pypi.org/simple/ gym-OT -q

In [1]:
import gym
from tqdm import tqdm

In [2]:
#env = gym.make("OT-v0")

# from ray.tune.registry import register_env
# import gym_OT
# select_env = "OT-v0"
# register_env(select_env, lambda config: OTGym())

In [6]:
import ray
ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8,num_gpus=1)

2022-11-17 18:39:24,731	INFO worker.py:1528 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.1.0


In [10]:
!rllib train --run DQN --env CartPole-v0 --framework torch

2022-11-17 18:46:18,302	WARNING utils.py:604 -- Ray currently does not support initializing Raywith fractional cpus. Your num_cpus will be truncated from 5.985 to 5.
2022-11-17 18:46:18,509	INFO worker.py:1528 -- Started a local Ray instance.
(DQN pid=20084) 2022-11-17 18:46:24,897	INFO simple_q.py:307 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQN pid=20084) 2022-11-17 18:46:24,900	INFO algorithm.py:457 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQN pid=20084) /usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
(DQN pid=20084)   logger.warn(
(DQN pid=20084) 2022-11-17 18:46:24,947	WARNING env.py:159 -- Your env reset() method appear

In [7]:


from ray.rllib.algorithms.dqn import DQN

# Configure the algorithm.
config = {
    "env": "Taxi-v3",
    "num_workers": 7,
    "framework": "torch",
    "num_gpus" : 1,
    "model": {
        "fcnet_hiddens": [640, 640,300],
        "fcnet_activation": "relu",
    },
    "evaluation_num_workers": 1,
    "evaluation_interval":1,
    "evaluation_config": {
        "render_env": False,
    },
}

algo = DQN(config=config)

# for it in tqdm(range(30)):
#     algo.train()
#     print(it,'  ',algo.evaluate()['evaluation']['episode_reward_mean'])


# algo.evaluate()


(RolloutWorker pid=14127) 2022-11-17 18:39:32,920	WARNING env.py:159 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2022-11-17 18:39:33,559	WARNING deprecation.py:47 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=14319) 2022-11-17 18:39:37,491	WARNING env.py:159 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2022-11-17 18:39:37,592	INFO trainable.py:164 -- Trainable.setup took 10.994 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [17]:
tune.TuneConfig

ray.tune.tune_config.TuneConfig

In [18]:
!pip3 install -U 'mujoco-py<2.2,>=2.1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 kB 38.5 MB/s eta 0:00:00


In [24]:
ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8,num_gpus=1)

import random

from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining

if __name__ == "__main__":

    # Postprocess the perturbed config to ensure it's still valid
    def explore(config):
        # ensure we collect enough timesteps to do sgd
        if config["train_batch_size"] < config["sgd_minibatch_size"] * 2:
            config["train_batch_size"] = config["sgd_minibatch_size"] * 2
        # ensure we run at least one sgd iter
        if config["num_sgd_iter"] < 1:
            config["num_sgd_iter"] = 1
        return config

    pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        
        # Specifies the mutations of these hyperparams
        hyperparam_mutations={
            "lambda": lambda: random.uniform(0.9, 1.0),
            # "clip_param": lambda: random.uniform(0.01, 0.5),
            "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "num_sgd_iter": lambda: random.randint(1, 30),
            "sgd_minibatch_size": lambda: random.randint(128, 16384),
            "train_batch_size": lambda: random.randint(2000, 160000),
        },
        custom_explore_fn=explore,
    )
    
    tuner = tune.Tuner(
        "DQN",
        tune_config=tune.TuneConfig(
            metric="episode_reward_mean",
            mode="max",
            scheduler=pbt,
            num_samples=1,
            reuse_actors=True,
            
            
        ),
        
        param_space={
            "env": "Taxi-v3",
            "framework": "torch",
            "num_workers": 7,
            "num_gpus": 1, # number of GPUs to use
            "model": {"free_log_std": True,
                      "fcnet_hiddens": [64, 64],
                      "fcnet_activation": "relu",
                     },
            # These params are tuned from a fixed starting value.
            # "lambda": 0.95,
            "clip_param": 0.2,
            "lr": 1e-4,
            # These params start off randomly drawn from a set.
            "num_sgd_iter": tune.choice([10, 20, 30]),
            "sgd_minibatch_size": tune.choice([128, 512, 2048]),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
            
        },
        
    )
    results = tuner.fit()

    print("best hyperparameters: ", results.get_best_result().config)

2022-11-17 17:25:21,548	INFO worker.py:1528 -- Started a local Ray instance.


2022-11-17 17:25:28,896	ERROR trial_runner.py:993 -- Trial DQN_Taxi-v3_d1bc5_00000: Error processing event.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/usr/local/lib/python3.8/dist-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ray/_private/worker.py", line 2291, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=7783, ip=10.42.32.217, repr=DQN)
  File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
    super().__init__(config=config, logger_creator=logger_creator, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/trainable

Trial name,trial_id
DQN_Taxi-v3_d1bc5_00000,d1bc5_00000


(DQN pid=7783) 2022-11-17 17:25:28,873	ERROR worker.py:763 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=7783, ip=10.42.32.217, repr=DQN)
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
(DQN pid=7783)     super().__init__(config=config, logger_creator=logger_creator, **kwargs)
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/tune/trainable/trainable.py", line 161, in __init__
(DQN pid=7783)     self.setup(copy.deepcopy(self.config))
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 439, in setup
(DQN pid=7783)     self.config = self.merge_trainer_configs(
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 2228, in merge_trainer_configs
(DQN pid=7783)     return deep_update(
(DQN pid=7783) Exception: Unknown config p

best hyperparameters:  None


In [7]:
a = algo.evaluate()


-532.1